# Wiener Linien Realtime API (Notebook)

This notebook starts a FastAPI proxy inside the Jupyter container.

- Set `FETCH_RBLS` in `docker-compose.yml` to enable auto-fetch.
- Run the cells to start the API on port 8000.
- Use the stop cell to shut down the API without stopping Jupyter.
- Access from host: http://localhost:8000/health


In [1]:
import sys
from pathlib import Path

candidates = [
    Path("/home/jovyan/api"),
    Path.cwd().parent / "api",
    Path.cwd() / "api",
]

api_root = next((p for p in candidates if (p / "app" / "main.py").exists()), None)
if api_root is None:
    raise FileNotFoundError("Could not find api/app/main.py. Check your volume mount or repo path.")

sys.path.insert(0, str(api_root))

from app.main import app


In [2]:
import threading
import uvicorn

def _build_server():
    config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
    return uvicorn.Server(config)

if "_api_thread" not in globals() or not _api_thread.is_alive():
    _api_server = _build_server()
    _api_thread = threading.Thread(target=_api_server.run, daemon=True)
    _api_thread.start()

print("API running at http://localhost:8000")


API running at http://localhost:8000


INFO:     Started server process [164]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


In [7]:
if "_api_server" in globals():
    _api_server.should_exit = True
    print("API stopping...")
else:
    print("API not running")


API stopping...


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [101]


In [3]:
import sys
!{sys.executable} -m pip install "pymongo==4.10.1"


In [3]:
print("ping")


ping


In [4]:
import os
print("FETCH_ENABLED:", os.getenv("FETCH_ENABLED"))
print("FETCH_RBLS:", os.getenv("FETCH_RBLS"))
print("MONGO_URI:", os.getenv("MONGO_URI"))
print("WL_BASE_URL:", os.getenv("WL_BASE_URL"))


FETCH_ENABLED: true
FETCH_RBLS: 147,148
MONGO_URI: mongodb://mongodb:27017
WL_BASE_URL: https://www.wienerlinien.at/ogd_realtime/


In [5]:
import httpx, os
url = os.getenv("WL_BASE_URL") + "monitor"
resp = httpx.get(url, params={"rbl": "147"}, timeout=10)
print(resp.status_code)
print(resp.text[:200])


200
{"data":{"monitors":[{"locationStop":{"type":"Feature","geometry":{"type":"Point","coordinates":[16.3492933,48.2248218]},"properties":{"name":"60201510","title":"Währinger Straße, Volksoper U","munici


In [6]:
from pymongo import MongoClient
client = MongoClient("mongodb://mongodb:27017")
col = client["big_data_austria"]["wienerlinien_realtime"]
print("count before:", col.count_documents({}))
col.insert_one({"test": True})
print("count after:", col.count_documents({}))


count before: 23
count after: 24
